In [2]:
import pandas as pd

#load the data
df = pd.read_csv('C:/Users/hemloka/Documents/de_project/raw/bakery_sales_revised.csv')

In [3]:
# ==== inspection ====

df.head()      # shows first 5 rows

,Transaction,Item,date_time,period_day,weekday_weekend
0,1,Bread,10/30/2016 9:58,morning,weekend
1,2,Scandinavian,10/30/2016 10:05,morning,weekend
2,2,Scandinavian,10/30/2016 10:05,morning,weekend
3,3,Hot chocolate,10/30/2016 10:07,morning,weekend
4,3,Jam,10/30/2016 10:07,morning,weekend


In [4]:
df.tail()      # shows last 5 rows

,Transaction,Item,date_time,period_day,weekday_weekend
20502,9682,Coffee,4/9/2017 14:32,afternoon,weekend
20503,9682,Tea,4/9/2017 14:32,afternoon,weekend
20504,9683,Coffee,4/9/2017 14:57,afternoon,weekend
20505,9683,Pastry,4/9/2017 14:57,afternoon,weekend
20506,9684,Smoothies,4/9/2017 15:04,afternoon,weekend


In [5]:
df.describe()   #shows numeric columns summary

,Transaction
count,20507.000000
mean,4976.202370
std,2796.203001
min,1.000000
25%,2552.000000
50%,5137.000000
75%,7357.000000
max,9684.000000


In [6]:
df.shape[1] #col count


5

In [7]:
df.shape  #row, col

(20507, 5)

In [8]:
df.isnull().mean()   #null % per col


Transaction        0.0
Item               0.0
date_time          0.0
period_day         0.0
weekday_weekend    0.0
dtype: float64

In [9]:
df.dtypes

Transaction         int64
Item               object
date_time          object
period_day         object
weekday_weekend    object
dtype: object

In [10]:
df["Item"].unique()
df["period_day"].unique()
df["weekday_weekend"].unique()




array(['weekend', 'weekday'], dtype=object)

In [11]:
# ==== CLEANING ====
# Convert datatypes
pd.to_datetime(df["date_time"])



0       2016-10-30 09:58:00
1       2016-10-30 10:05:00
2       2016-10-30 10:05:00
3       2016-10-30 10:07:00
4       2016-10-30 10:07:00
                ...        
20502   2017-04-09 14:32:00
20503   2017-04-09 14:32:00
20504   2017-04-09 14:57:00
20505   2017-04-09 14:57:00
20506   2017-04-09 15:04:00
Name: date_time, Length: 20507, dtype: datetime64[ns]

In [12]:
#standardize string columns
df["Item"] = df["Item"].str.title()
df['Item'].unique()


array(['Bread', 'Scandinavian', 'Hot Chocolate', 'Jam', 'Cookies',
       'Muffin', 'Coffee', 'Pastry', 'Medialuna', 'Tea', 'Tartine',
       'Basket', 'Mineral Water', 'Farm House', 'Fudge', 'Juice',
       "Ella'S Kitchen Pouches", 'Victorian Sponge', 'Frittata',
       'Hearty & Seasonal', 'Soup', 'Pick And Mix Bowls', 'Smoothies',
       'Cake', 'Mighty Protein', 'Chicken Sand', 'Coke',
       'My-5 Fruit Shoot', 'Focaccia', 'Sandwich', 'Alfajores', 'Eggs',
       'Brownie', 'Dulce De Leche', 'Honey', 'The Bart', 'Granola',
       'Fairy Doors', 'Empanadas', 'Keeping It Local', 'Art Tray',
       'Bowl Nic Pitt', 'Bread Pudding', 'Adjustment', 'Truffles',
       'Chimichurri Oil', 'Bacon', 'Spread', 'Kids Biscuit', 'Siblings',
       'Caramel Bites', 'Jammie Dodgers', 'Tiffin', 'Olum & Polenta',
       'Polenta', 'The Nomad', 'Hack The Stack', 'Bakewell',
       'Lemon And Coconut', 'Toast', 'Scone', 'Crepes', 'Vegan Mincepie',
       'Bare Popcorn', 'Muesli', 'Crisps', 'Pintxos', 

In [13]:
##Strip whitespace

str_cols = df.select_dtypes(include="object").columns

# Create a report
whitespace_report = df[str_cols].apply(lambda x: x.str.match(r'^\s|\s$')).sum()
print(whitespace_report)

#lambda verison
for col in str_cols:
    count_spaces = (df[col].str.startswith(' ') | df[col].str.endswith(' ')).sum()
    print(f"{col}: {count_spaces} rows with extra spaces")


Item               0
date_time          0
period_day         0
weekday_weekend    0
dtype: int64
Item: 0 rows with extra spaces
date_time: 0 rows with extra spaces
period_day: 0 rows with extra spaces
weekday_weekend: 0 rows with extra spaces


In [14]:
df['weekday_weekend'].unique()

array(['weekend', 'weekday'], dtype=object)

In [17]:
#Add invalid_flag for rows failing basic validation (e.g., negative sales, missing customer_id)

df['invalid_flag'] = df.apply(
    lambda row: 1 if (row['Transaction'] < 0 
                      or pd.isnull(row['period_day'])) 
            else 0, 
    axis=1
)
df


,Transaction,Item,date_time,period_day,weekday_weekend,invalid_flag
0,1,Bread,10/30/2016 9:58,morning,weekend,0
1,2,Scandinavian,10/30/2016 10:05,morning,weekend,0
2,2,Scandinavian,10/30/2016 10:05,morning,weekend,0
3,3,Hot Chocolate,10/30/2016 10:07,morning,weekend,0
4,3,Jam,10/30/2016 10:07,morning,weekend,0
...,...,...,...,...,...,...
20502,9682,Coffee,4/9/2017 14:32,afternoon,weekend,0
20503,9682,Tea,4/9/2017 14:32,afternoon,weekend,0
20504,9683,Coffee,4/9/2017 14:57,afternoon,weekend,0
20505,9683,Pastry,4/9/2017 14:57,afternoon,weekend,0


In [16]:
df.to_csv(r"C:\Users\hemloka\Documents\de_project\clean\day1_bakerysales.csv", index=False)



In [ ]:
df.groupby('Item').agg(
    total_orders=('Transaction', 'size'),
    missing_dates=('date_time', lambda x: x.isna().sum()),
    avg_transaction=('Transaction', 'mean'),
    weekend_orders=('weekday_weekend', lambda x: (x == 'weekend').sum()),
    invalid_records=('invalid_flag', 'sum')
)


,total_orders,missing_dates,avg_transaction,weekend_orders,invalid_records
Item,,,,,
Adjustment,1,0,938.000000,0,0
Afternoon With The Baker,44,0,6500.636364,26,0
Alfajores,369,0,4099.119241,137,0
Argentina Night,7,0,9164.714286,6,0
Art Tray,38,0,3707.289474,12,0
...,...,...,...,...,...
Tshirt,21,0,6010.761905,21,0
Valentine'S Card,13,0,6140.076923,7,0
Vegan Feast,16,0,8496.500000,12,0


In [ ]:
df.groupby("Item").count()


Item
Adjustment                    1
Afternoon With The Baker     44
Alfajores                   369
Argentina Night               7
Art Tray                     38
                           ... 
Tshirt                       21
Valentine'S Card             13
Vegan Feast                  16
Vegan Mincepie               54
Victorian Sponge              7
Length: 94, dtype: int64

In [52]:
import sqlite3
import pandas as pd
conn = sqlite3.connect("../sql/day1_bakery.db")


In [53]:
df.to_sql("bakery_clean", conn, if_exists="replace", index=False)
pd.read_sql_query("""
SELECT Item, COUNT(*) AS total_orders
FROM bakery_clean
GROUP BY Item
ORDER BY total_orders DESC;
""", conn)



,Item,total_orders
0,Coffee,5471
1,Bread,3325
2,Tea,1435
3,Cake,1025
4,Pastry,856
...,...,...
89,Olum & Polenta,1
90,Gift Voucher,1
91,Chicken Sand,1
92,Bacon,1
